# Mental Health Classification with Neural Network

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import seaborn as sns
import math
import tensorflow as tf
import keras
import keras_tuner as kt

from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report

from tensorflow.keras import callbacks
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense, Activation, Dropout, InputLayer, Input, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Accuracy
from keras import layers, Sequential
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import scale, OneHotEncoder, StandardScaler
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report
from keras import backend as K

In [2]:
df = pd.read_csv('../datasets/mentalhealth.csv')
# df = df.iloc[0:3000]
df.head()

,Timestamp,Gender,Country,Occupation,self_employed,family_history,treatment,Days_Indoors,Growing_Stress,Changes_Habits,Mental_Health_History,Mood_Swings,Coping_Struggles,Work_Interest,Social_Weakness,mental_health_interview,care_options
0,8/27/2014 11:29,Female,United States,Corporate,NaN,No,Yes,1-14 days,Yes,No,Yes,Medium,No,No,Yes,No,Not sure
1,8/27/2014 11:31,Female,United States,Corporate,NaN,Yes,Yes,1-14 days,Yes,No,Yes,Medium,No,No,Yes,No,No
2,8/27/2014 11:32,Female,United States,Corporate,NaN,Yes,Yes,1-14 days,Yes,No,Yes,Medium,No,No,Yes,No,Yes
3,8/27/2014 11:37,Female,United States,Corporate,No,Yes,Yes,1-14 days,Yes,No,Yes,Medium,No,No,Yes,Maybe,Yes
4,8/27/2014 11:43,Female,United States,Corporate,No,Yes,Yes,1-14 days,Yes,No,Yes,Medium,No,No,Yes,No,Yes


In [3]:
df.drop(['Timestamp', 'Gender', 'Country'], axis=1, inplace=True)

In [4]:
df.head()

,Occupation,self_employed,family_history,treatment,Days_Indoors,Growing_Stress,Changes_Habits,Mental_Health_History,Mood_Swings,Coping_Struggles,Work_Interest,Social_Weakness,mental_health_interview,care_options
0,Corporate,NaN,No,Yes,1-14 days,Yes,No,Yes,Medium,No,No,Yes,No,Not sure
1,Corporate,NaN,Yes,Yes,1-14 days,Yes,No,Yes,Medium,No,No,Yes,No,No
2,Corporate,NaN,Yes,Yes,1-14 days,Yes,No,Yes,Medium,No,No,Yes,No,Yes
3,Corporate,No,Yes,Yes,1-14 days,Yes,No,Yes,Medium,No,No,Yes,Maybe,Yes
4,Corporate,No,Yes,Yes,1-14 days,Yes,No,Yes,Medium,No,No,Yes,No,Yes


## Mencari info dari data

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 292364 entries, 0 to 292363
Data columns (total 14 columns):
 #   Column                   Non-Null Count   Dtype 
---  ------                   --------------   ----- 
 0   Occupation               292364 non-null  object
 1   self_employed            287162 non-null  object
 2   family_history           292364 non-null  object
 3   treatment                292364 non-null  object
 4   Days_Indoors             292364 non-null  object
 5   Growing_Stress           292364 non-null  object
 6   Changes_Habits           292364 non-null  object
 7   Mental_Health_History    292364 non-null  object
 8   Mood_Swings              292364 non-null  object
 9   Coping_Struggles         292364 non-null  object
 10  Work_Interest            292364 non-null  object
 11  Social_Weakness          292364 non-null  object
 12  mental_health_interview  292364 non-null  object
 13  care_options             292364 non-null  object
dtypes: object(14)
memory

In [6]:
df['self_employed'].dropna(inplace=True)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 292364 entries, 0 to 292363
Data columns (total 14 columns):
 #   Column                   Non-Null Count   Dtype 
---  ------                   --------------   ----- 
 0   Occupation               292364 non-null  object
 1   self_employed            287162 non-null  object
 2   family_history           292364 non-null  object
 3   treatment                292364 non-null  object
 4   Days_Indoors             292364 non-null  object
 5   Growing_Stress           292364 non-null  object
 6   Changes_Habits           292364 non-null  object
 7   Mental_Health_History    292364 non-null  object
 8   Mood_Swings              292364 non-null  object
 9   Coping_Struggles         292364 non-null  object
 10  Work_Interest            292364 non-null  object
 11  Social_Weakness          292364 non-null  object
 12  mental_health_interview  292364 non-null  object
 13  care_options             292364 non-null  object
dtypes: object(14)
memory

In [8]:
df.columns

Index(['Occupation', 'self_employed', 'family_history', 'treatment',
       'Days_Indoors', 'Growing_Stress', 'Changes_Habits',
       'Mental_Health_History', 'Mood_Swings', 'Coping_Struggles',
       'Work_Interest', 'Social_Weakness', 'mental_health_interview',
       'care_options'],
      dtype='object')

In [9]:
df['treatment'].value_counts()

Yes    147606
No     144758
Name: treatment, dtype: int64

In [10]:
df.replace({
    'self_employed': {
        'Yes': 1,
        'No': 0
    },
    'family_history': {
        'Yes': 1,
        'No': 0
    },
    'treatment': {
        'Yes': 1,
        'No': 0
    },
    'Days_Indoors': {
        'Go out Every day': 1,
        '1-14 days': 2,
        '15-30 days': 3,
        '31-60 days': 4,
        'More than 2 months': 5
    },
    'Growing_Stress': {
        'Yes': 3,
        'Maybe': 2,
        'No': 1
    },
    'Changes_Habits': {
        'Yes': 3,
        'Maybe': 2,
        'No': 1
    },
    'Mental_Health_History': {
        'Yes': 3,
        'Maybe': 2,
        'No': 1
    },
    'Mood_Swings': {
        'Medium': 2,
        'Low': 1,
        'High': 3
    },
    'Coping_Struggles': {
        'Yes': 1,
        'No': 0
    },
    'Work_Interest': {
        'Yes': 3,
        'Maybe': 2,
        'No': 1
    },
    'Social_Weakness': {
        'Yes': 3,
        'No': 1,
        'Maybe': 2
    },
    'mental_health_interview': {
        'Yes': 3,
        'No': 1,
        'Maybe': 2
    },
    'care_options': {
        'Not sure': 2,
        'No': 1,
        'Yes': 3,
        'Ye': 3
    }
}, inplace=True)

In [11]:
for column in df.columns:
    print(f"Column Name: {column} - {df[column].unique()}")

Column Name: Occupation - ['Corporate' 'Student' 'Business' 'Housewife' 'Others']
Column Name: self_employed - [nan  0.  1.]
Column Name: family_history - [0 1]
Column Name: treatment - [1 0]
Column Name: Days_Indoors - [2 1 5 3 4]
Column Name: Growing_Stress - [3 1 2]
Column Name: Changes_Habits - [1 3 2]
Column Name: Mental_Health_History - [3 1 2]
Column Name: Mood_Swings - [2 1 3]
Column Name: Coping_Struggles - [0 1]
Column Name: Work_Interest - [1 2 3]
Column Name: Social_Weakness - [3 1 2]
Column Name: mental_health_interview - [1 2 3]
Column Name: care_options - [2 1 3]


In [12]:
df.tail()

,Occupation,self_employed,family_history,treatment,Days_Indoors,Growing_Stress,Changes_Habits,Mental_Health_History,Mood_Swings,Coping_Struggles,Work_Interest,Social_Weakness,mental_health_interview,care_options
292359,Business,1.0,1,1,3,1,2,1,1,1,1,2,2,2
292360,Business,0.0,1,1,3,1,2,1,1,1,1,2,1,3
292361,Business,0.0,1,0,3,1,2,1,1,1,1,2,1,1
292362,Business,0.0,1,1,3,1,2,1,1,1,1,2,1,3
292363,Business,0.0,1,1,3,1,2,1,1,1,1,2,1,3


In [136]:
df = pd.get_dummies(df, columns=['Occupation'])
df.tail()

,self_employed,family_history,treatment,Days_Indoors,Growing_Stress,Changes_Habits,Mental_Health_History,Mood_Swings,Coping_Struggles,Work_Interest,Social_Weakness,mental_health_interview,care_options,Occupation_Business,Occupation_Corporate,Occupation_Housewife,Occupation_Others,Occupation_Student
292359,1.0,1,1,3,1,2,1,1,1,1,2,2,2,1,0,0,0,0
292360,0.0,1,1,3,1,2,1,1,1,1,2,1,3,1,0,0,0,0
292361,0.0,1,0,3,1,2,1,1,1,1,2,1,1,1,0,0,0,0
292362,0.0,1,1,3,1,2,1,1,1,1,2,1,3,1,0,0,0,0
292363,0.0,1,1,3,1,2,1,1,1,1,2,1,3,1,0,0,0,0


In [15]:
X = df.drop(['treatment', 'Occupation'], axis=1)
y = df['treatment']

## Split the Data

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)

X_train = scale(X_train)
X_test = scale(X_test)

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

print(f"""
X_train shape = {X_train.shape}
y_train shape = {y_train.shape}
X_test shape = {X_test.shape}
y_test shape = {y_test.shape}
""")


X_train shape = (233891, 12)
y_train shape = (233891, 2)
X_test shape = (58473, 12)
y_test shape = (58473, 2)



## Create Model

In [103]:
def build_model(hp):
    model = Sequential()
    counter=0
    model.add(Input((17,)))
    for i in range(hp.Int('num_layers',min_value=1,max_value=16)):
        model.add(Dense(hp.Int('units'+str(i),min_value=8,max_value=128,step=4),activation=hp.Choice('activation' + str(i),values=['relu'])))
        model.add(Dropout(hp.Choice('drp'+str(i),values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
    model.add(Dense(2, activation='softmax'))
    
    model.compile(optimizer=hp.Choice('optimizer',values=['adam', 'nadam']),loss='binary_crossentropy',metrics=['accuracy'])
    return model

In [104]:
tuner=kt.RandomSearch(build_model,objective='val_accuracy',max_trials=50)

In [105]:
tuner.search(X_train,y_train,epochs=5,validation_split=0.2)
tuner.get_best_hyperparameters()[0].values

Trial 50 Complete [00h 00m 19s]
val_accuracy: 0.5669089555740356

Best val_accuracy So Far: 0.7092774510383606
Total elapsed time: 00h 16m 42s


{'num_layers': 2,
 'units0': 124,
 'activation0': 'relu',
 'drp0': 0.2,
 'optimizer': 'nadam',
 'units1': 108,
 'activation1': 'relu',
 'drp1': 0.2,
 'units2': 44,
 'activation2': 'relu',
 'drp2': 0.8,
 'units3': 100,
 'activation3': 'relu',
 'drp3': 0.2,
 'units4': 68,
 'activation4': 'relu',
 'drp4': 0.9,
 'units5': 96,
 'activation5': 'relu',
 'drp5': 0.1,
 'units6': 80,
 'activation6': 'relu',
 'drp6': 0.2,
 'units7': 56,
 'activation7': 'relu',
 'drp7': 0.5,
 'units8': 36,
 'activation8': 'relu',
 'drp8': 0.1,
 'units9': 84,
 'activation9': 'relu',
 'drp9': 0.9,
 'units10': 84,
 'activation10': 'relu',
 'drp10': 0.3,
 'units11': 68,
 'activation11': 'relu',
 'drp11': 0.3,
 'units12': 92,
 'activation12': 'relu',
 'drp12': 0.9,
 'units13': 60,
 'activation13': 'relu',
 'drp13': 0.6,
 'units14': 124,
 'activation14': 'relu',
 'drp14': 0.8,
 'units15': 128,
 'activation15': 'relu',
 'drp15': 0.5}

In [27]:
def model_builder(hp):
    model = Sequential()
    model.add(Input((12,)))

    hp_units = hp.Int('units', min_value=8, max_value=512, step=8)
    model.add(Dense(units=hp_units, activation='relu'))
    model.add(Dense(2, activation='softmax'))

    hp_learning_rate = hp.Choice('learning_rate', values=[0.1, 0.01, 0.001, 0.0001])

    model.compile(optimizer=Adam(learning_rate=hp_learning_rate), loss='binary_crossentropy', metrics=['accuracy'])

    return model

In [28]:
tuner = kt.Hyperband(model_builder, 
                     objective='val_accuracy', 
                     max_epochs=10,
                     factor=3,
                     directory='backpro_mentalhealth')

Reloading Tuner from backpro_mentalhealth\untitled_project\tuner0.json


In [29]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [30]:
tuner.search(X_train, y_train, epochs=50, validation_split=0.2, callbacks=[stop_early], batch_size=1000)

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

Trial 53 Complete [00h 00m 19s]
val_accuracy: 0.5089462995529175

Best val_accuracy So Far: 0.7195382714271545
Total elapsed time: 00h 03m 22s


In [ ]:
model = tuner.hypermodel.build(best_hps)
history = model.fit(X_train, y_train, epochs=50, validation_split=0.2, batch_size=100)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch:', best_epoch)

Epoch 1/50
1872/1872 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - accuracy: 0.5005 - loss: 0.6936 - val_accuracy: 0.4911 - val_loss: 0.6936
Epoch 2/50
1872/1872 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.4997 - loss: 0.6933 - val_accuracy: 0.4911 - val_loss: 0.6936
Epoch 3/50
1847/1872 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4998 - loss: 0.6933

In [ ]:
hypermodel = tuner.hypermodel.build(best_hps)
hypermodel.fit(X_train, y_train, epochs=best_epoch, validation_split=0)

In [119]:
eval_result = hypermodel.evaluate(X_test, y_test)
print("[test loss, test accuracy]:", eval_result)

92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7068 - loss: 0.5851
[test loss, test accuracy]: [0.5769459009170532, 0.7181942462921143]


In [85]:
model=tuner.get_best_models(num_models=1)[0]
model.summary()

C:\Users\itzbi\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\saving\saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'nadam', because it has 2 variables whereas the saved optimizer has 19 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 36)                  │             648 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 36)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 34)                  │           1,258 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 34)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 8)                   │             280 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 8)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 2)                   │              18 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,204 (8.61 KB)

 Trainable params: 2,204 (8.61 KB)

 Non-trainable params: 0 (0.00 B)

In [86]:
new_model = Sequential([
    Input((17,)),
    Dense(36, activation='relu'),
    Dropout(0.4),
    Dense(34, activation='relu'),
    Dropout(0.1),
    Dense(8, activation='relu'),
    Dropout(0.3),
    Dense(2, activation='softmax'),
])

new_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

new_model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_4 (Dense)                      │ (None, 36)                  │             648 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 36)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 34)                  │           1,258 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 34)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 8)                   │             280 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_5 (Dropout)                  │ (None, 8)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 2)                   │              18 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,204 (8.61 KB)

 Trainable params: 2,204 (8.61 KB)

 Non-trainable params: 0 (0.00 B)

In [87]:
new_model.fit(X_train, y_train, validation_split=0.2, epochs=50)

Epoch 1/50
293/293 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.5502 - loss: 0.6940 - val_accuracy: 0.6704 - val_loss: 0.6266
Epoch 2/50
293/293 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.6472 - loss: 0.6368 - val_accuracy: 0.6789 - val_loss: 0.6108
Epoch 3/50
293/293 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.6739 - loss: 0.6283 - val_accuracy: 0.6952 - val_loss: 0.6010
Epoch 4/50
293/293 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.6685 - loss: 0.6249 - val_accuracy: 0.6952 - val_loss: 0.6004
Epoch 5/50
293/293 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.6758 - loss: 0.6177 - val_accuracy: 0.7003 - val_loss: 0.5964
Epoch 6/50
293/293 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.6897 - loss: 0.6167 - val_accuracy: 0.6965 - val_loss: 0.5960
Epoch 7/50
293/293 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.6834 - loss: 0.6153 - val_accuracy: 0.6977 - val_loss: 0.5931
Epoch 8/50
293/293 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6969 - loss: 0.6058 - val_accuracy: 0.